In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.ensemble import GradientBoostingClassifier

import plotly.express as px  # for data visualization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_matrix = pd.read_csv('/content/drive/My Drive/CSC373_DMP_Mayr_Jack/DMP_Final/nasa.csv')
data_matrix

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),Est Dia in Feet(max),Close Approach Date,Epoch Date Close Approach,Relative Velocity km per sec,Relative Velocity km per hr,Miles per hour,Miss Dist.(Astronomical),Miss Dist.(lunar),Miss Dist.(kilometers),Miss Dist.(miles),Orbiting Body,Orbit ID,Orbit Determination Date,Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Epoch Osculation,Eccentricity,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,3703080,3703080,21.600,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,933.308089,1995-01-01,788947200000,6.115834,22017.003799,13680.509944,0.419483,163.178711,6.275369e+07,3.899334e+07,Earth,17,2017-04-06 08:36:37,5,0.025282,4.634,2458000.5,0.425549,1.407011,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,J2000,True
1,3723955,3723955,21.300,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,1071.581063,1995-01-01,788947200000,18.113985,65210.346095,40519.173105,0.383014,148.992630,5.729815e+07,3.560342e+07,Earth,21,2017-04-06 08:32:49,3,0.186935,5.457,2458000.5,0.351674,1.107776,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,J2000,False
2,2446862,2446862,20.300,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,1698.341531,1995-01-08,789552000000,7.590711,27326.560182,16979.661798,0.050956,19.821890,7.622912e+06,4.736658e+06,Earth,22,2017-04-06 09:20:19,0,0.043058,4.557,2458000.5,0.348248,1.458824,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,J2000,True
3,3092506,3092506,27.400,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,64.569144,1995-01-15,790156800000,11.173874,40225.948191,24994.839864,0.285322,110.990387,4.268362e+07,2.652237e+07,Earth,7,2017-04-06 09:15:49,6,0.005512,5.093,2458000.5,0.216578,1.255903,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,J2000,False
4,3514799,3514799,21.600,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,933.308089,1995-01-15,790156800000,9.840831,35426.991794,22012.954985,0.407832,158.646713,6.101082e+07,3.791037e+07,Earth,25,2017-04-06 08:57:58,1,0.034798,5.154,2458000.5,0.210448,1.225615,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,J2000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,3759007,3759007,23.900,0.044112,0.098637,44.111820,98.637028,0.027410,0.061290,144.723824,323.612307,2016-09-08,1473318000000,22.154265,79755.354273,49556.875548,0.041361,16.089413,6.187511e+06,3.844741e+06,Earth,4,2017-04-06 08:24:07,8,0.019777,5.156,2457637.5,0.361512,1.161429,39.880491,164.183305,457.179984,0.741558,276.395697,1.581299,2.457708e+06,304.306025,0.787436,J2000,False
4683,3759295,3759295,28.200,0.006089,0.013616,6.089126,13.615700,0.003784,0.008460,19.977449,44.670934,2016-09-08,1473318000000,3.225150,11610.539577,7214.337772,0.006469,2.516399,9.677324e+05,6.013211e+05,Earth,2,2017-04-06 08:23:58,6,0.006451,5.742,2458000.5,0.073200,1.075134,5.360249,345.225230,407.185767,0.996434,42.111064,1.153835,2.458088e+06,282.978786,0.884117,J2000,False
4684,3759714,3759714,22.700,0.076658,0.171412,76.657557,171.411509,0.047633,0.106510,251.501180,562.373736,2016-09-08,1473318000000,7.191642,25889.910626,16086.983633,0.061009,23.732393,9.126775e+06,5.671115e+06,Earth,17,2017-04-06 08:23:43,6,0.059973,4.410,2458000.5,0.368055,1.528234,4.405467,37.026468,690.054279,0.965760,274.692712,2.090708,2.458300e+06,203.501147,0.521698,J2000,False
4685,37597

In [ ]:
data_matrix = data_matrix.drop(columns = ['Miss Dist.(kilometers)', 'Orbit ID', 'Neo Reference ID', 'Name', 'Orbit Determination Date', 'Close Approach Date',  'Epoch Date Close Approach' ,'Est Dia in KM(min)', 'Est Dia in KM(max)', 'Est Dia in M(min)', 'Est Dia in M(max)', 'Est Dia in Miles(min)', 'Est Dia in Miles(max)', 'Orbiting Body', 'Epoch Osculation','Equinox'])

In [ ]:
data_matrix.loc[data_matrix['Hazardous']== True, 'Hazardous'] = 1 
data_matrix.loc[data_matrix['Hazardous']== False, 'Hazardous'] = 0 
data_matrix

,Absolute Magnitude,Est Dia in Feet(min),Est Dia in Feet(max),Relative Velocity km per sec,Relative Velocity km per hr,Miles per hour,Miss Dist.(Astronomical),Miss Dist.(lunar),Miss Dist.(miles),Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Eccentricity,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Hazardous
0,21.600,417.388066,933.308089,6.115834,22017.003799,13680.509944,0.419483,163.178711,3.899334e+07,5,0.025282,4.634,0.425549,1.407011,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,1
1,21.300,479.225620,1071.581063,18.113985,65210.346095,40519.173105,0.383014,148.992630,3.560342e+07,3,0.186935,5.457,0.351674,1.107776,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,0
2,20.300,759.521423,1698.341531,7.590711,27326.560182,16979.661798,0.050956,19.821890,4.736658e+06,0,0.043058,4.557,0.348248,1.458824,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,1
3,27.400,28.876199,64.569144,11.173874,40225.948191,24994.839864,0.285322,110.990387,2.652237e+07,6,0.005512,5.093,0.216578,1.255903,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,0
4,21.600,417.388066,933.308089,9.840831,35426.991794,22012.954985,0.407832,158.646713,3.791037e+07,1,0.034798,5.154,0.210448,1.225615,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,23.900,144.723824,323.612307,22.154265,79755.354273,49556.875548,0.041361,16.089413,3.844741e+06,8,0.019777,5.156,0.361512,1.161429,39.880491,164.183305,457.179984,0.741558,276.395697,1.581299,2.457708e+06,304.306025,0.787436,0
4683,28.200,19.977449,44.670934,3.225150,11610.539577,7214.337772,0.006469,2.516399,6.013211e+05,6,0.006451,5.742,0.073200,1.075134,5.360249,345.225230,407.185767,0.996434,42.111064,1.153835,2.458088e+06,282.978786,0.884117,0
4684,22.700,251.501180,562.373736,7.191642,25889.910626,16086.983633,0.061009,23.732393,5.671115e+06,6,0.059973,4.410,0.368055,1.528234,4.405467,37.026468,690.054279,0.965760,274.692712,2.090708,2.458300e+06,203.501147,0.521698,0
4685,21.800,380.662441,851.187094,11.352090,40867.522309,25393.489071,0.260760,101.435493,2.423912e+07,5,0.177510,4.477,0.202565,1.486600,21.080244,163.802910,662.048343,1.185467,180.346090,1.787733,2.458288e+06,203.524965,0.543767,0


In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float32)

In [ ]:
data_matrix = clean_dataset(data_matrix)
data_matrix

,Absolute Magnitude,Est Dia in Feet(min),Est Dia in Feet(max),Relative Velocity km per sec,Relative Velocity km per hr,Miles per hour,Miss Dist.(Astronomical),Miss Dist.(lunar),Miss Dist.(miles),Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Eccentricity,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Hazardous
0,21.600000,417.388062,933.308105,6.115834,22017.003906,13680.509766,0.419483,163.178711,3.899334e+07,5.0,0.025282,4.634,0.425549,1.407011,6.025981,314.373901,609.599792,0.808259,57.257469,2.005764,2458161.75,264.837524,0.590551,1.0
1,21.299999,479.225616,1071.581055,18.113985,65210.347656,40519.171875,0.383014,148.992630,3.560342e+07,3.0,0.186935,5.457,0.351674,1.107776,28.412996,136.717239,425.869293,0.718200,313.091980,1.497352,2457795.00,173.741119,0.845330,0.0
2,20.299999,759.521423,1698.341553,7.590711,27326.560547,16979.662109,0.050956,19.821890,4.736658e+06,0.0,0.043058,4.557,0.348248,1.458824,4.237961,259.475983,643.580200,0.950791,248.415039,1.966857,2458120.50,292.893646,0.559371,1.0
3,27.400000,28.876200,64.569145,11.173875,40225.949219,24994.839844,0.285322,110.990387,2.652237e+07,6.0,0.005512,5.093,0.216578,1.255903,7.905894,57.173267,514.082153,0.983902,18.707701,1.527904,2457902.25,68.741005,0.700277,0.0
4,21.600000,417.388062,933.308105,9.840831,35426.992188,22012.955078,0.407832,158.646713,3.791037e+07,1.0,0.034798,5.154,0.210448,1.225615,16.793381,84.629311,495.597809,0.967687,158.263596,1.483543,2457814.50,135.142136,0.726395,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,23.900000,144.723816,323.612305,22.154264,79755.351562,49556.875000,0.041361,16.089413,3.844741e+06,8.0,0.019777,5.156,0.361512,1.161429,39.880489,164.183304,457.179993,0.741558,276.395691,1.581299,2457708.25,304.306030,0.787436,0.0
4683,28.200001,19.977449,44.670933,3.225150,11610.540039,7214.337891,0.006469,2.516399,6.013211e+05,6.0,0.006451,5.742,0.073200,1.075134,5.360249,345.225220,407.185760,0.996434,42.111065,1.153835,2458087.50,282.978790,0.884117,0.0
4684,22.700001,251.501175,562.373718,7.191642,25889.910156,16086.983398,0.061009,23.732393,5.671115e+06,6.0,0.059972,4.410,0.368055,1.528234,4.405467,37.026466,690.054260,0.965760,274.692719,2.090708,2458300.50,203.501144,0.521698,0.0
4685,21.799999,380.662445,851.187073,11.352090,40867.523438,25393.488281,0.260760,101.435493,2.423912e+07,5.0,0.177510,4.477,0.202565,1.486600,21.080244,163.802902,662.048340,1.185467,180.346085,1.787733,2458288.25,203.524963,0.543767,0.0


In [ ]:

def model_training(X, y, n_trees):


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    model = GradientBoostingClassifier(loss='deviance', criterion='squared_error', learning_rate=0.1, subsample=1.0, random_state=0, max_features='sqrt', min_samples_leaf=1000, max_depth=3, n_estimators=n_trees)

    clf = model.fit(X_train, y_train)

    training_prediction_labels = model.predict(X_train)
    testing_prediction_labels = model.predict(X_test)

    print('_________________ Ensemble Summary _________________')
    print('No. of classes: ', clf.n_classes_)
    print('Classes: ', clf.classes_)
    print('No. of features: ', clf.n_features_in_)
    print('No. of Estimators: ', len(clf.estimators_))
    print('____________________________________________________')
    print("")

    print('_______________ Testing Evaluation _________________')
    testing_score = model.score(X_test, y_test)
    print('Accuracy Score: ', testing_score)
    print(classification_report(y_test, testing_prediction_labels))
    print('____________________________________________________')
    print("")

    print('_______________ Training Evaluation ________________')
    training_score = model.score(X_train, y_train)
    print('Accuracy Score: ', training_score)
    print(classification_report(y_train, training_prediction_labels))
    print('_____________________________________________________')
    
    return clf, X_test, y_test

In [ ]:
X = data_matrix.drop(columns=['Hazardous'])
y = data_matrix['Hazardous']
clf, X_test, y_test = model_training(X, y, n_trees=1000)



_________________ Ensemble Summary _________________
No. of classes:  2
Classes:  [0. 1.]
No. of features:  23
No. of Estimators:  1000
____________________________________________________

_______________ Testing Evaluation _________________
Accuracy Score:  0.9936034115138592
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       793
         1.0       0.99      0.97      0.98       145

    accuracy                           0.99       938
   macro avg       0.99      0.98      0.99       938
weighted avg       0.99      0.99      0.99       938

____________________________________________________

_______________ Training Evaluation ________________
Accuracy Score:  1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3139
         1.0       1.00      1.00      1.00       610

    accuracy                           1.00      3749
   macro avg       1.00      1.00      1.00     

In [ ]:
fig = px.scatter_3d(data_matrix, x='Absolute Magnitude', y='Miss Dist.(kilometers)', z='Miles per hour',color='Hazardous')
fig.show()
  